# Projeto de Transfer Learning com Cats vs Dogs

Este notebook aplica o método de Transfer Learning utilizando o modelo MobileNetV2 para classificar imagens de gatos e cachorros. O dataset utilizado é o `cats_vs_dogs` disponível no TensorFlow Datasets e os dados baixados do link da Microsoft.

## Objetivos
- Utilizar Transfer Learning para classificar imagens em duas classes: gatos e cachorros.
- Demonstrar o uso de pré-processamento de imagens e treinamento de redes neurais.
- Visualizar os resultados do treinamento.

## Links úteis
- [Descrição do Dataset](https://www.tensorflow.org/datasets/catalog/cats_vs_dogs)
- [Download do Dataset](https://www.microsoft.com/en-us/download/details.aspx?id=54765)
- [Exemplo com MNIST](https://colab.research.google.com/github/kylemath/ml4a-guides/blob/master/notebooks/transfer-learning.ipynb)

In [ ]:
# Importação de Bibliotecas
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
# Download do Dataset Cats vs Dogs (Microsoft)
import zipfile
import requests

def download_and_extract_dataset(url, extract_to):
    response = requests.get(url)
    zip_path = os.path.join(extract_to, "cats_vs_dogs.zip")
    with open(zip_path, "wb") as f:
        f.write(response.content)
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Dataset extraído para {extract_to}")

# URL do dataset e diretório de extração
dataset_url = "https://www.microsoft.com/en-us/download/details.aspx?id=54765"
dataset_dir = "./cats_vs_dogs_data"
os.makedirs(dataset_dir, exist_ok=True)
download_and_extract_dataset(dataset_url, dataset_dir)

In [ ]:
# Carregar e pré-processar imagens do diretório extraído
def load_and_preprocess_custom_dataset(directory):
    dataset = image_dataset_from_directory(
        directory,
        image_size=(224, 224),
        batch_size=32
    )
    return dataset.map(lambda x, y: (x / 255.0, y))

custom_dataset = load_and_preprocess_custom_dataset(dataset_dir)
train_custom_ds = custom_dataset.take(int(len(custom_dataset) * 0.8))
test_custom_ds = custom_dataset.skip(int(len(custom_dataset) * 0.8))

In [ ]:
# Carregamento do Dataset (Cats vs Dogs)
# Corrigir o carregamento do dataset para garantir que as divisões de treino e teste funcionem corretamente
try:
    (dataset_train, dataset_test), info = tfds.load(
        'cats_vs_dogs',
        split=['train[:80%]', 'train[80%:]'],
        with_info=True,
        as_supervised=True
    )
    print("Dataset carregado com sucesso!")
except Exception as e:
    print(f"Erro ao carregar o dataset: {e}")

In [ ]:
# Pré-processamento das Imagens do TensorFlow Datasets
# Garantir que o dataset seja pré-processado corretamente
def preprocess(image, label):
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image, label

try:
    train_ds = dataset_train.map(preprocess).batch(32).prefetch(1)
    test_ds  = dataset_test.map(preprocess).batch(32).prefetch(1)
    print("Pré-processamento concluído com sucesso!")
except Exception as e:
    print(f"Erro no pré-processamento: {e}")

In [ ]:
# Combinar os datasets (TensorFlow e Microsoft)
train_combined_ds = train_ds.concatenate(train_custom_ds)
test_combined_ds = test_ds.concatenate(test_custom_ds)

In [ ]:
# Modelo de Transfer Learning com MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Congelar pesos

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Classificação binária
])

In [ ]:
# Compilação do Modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Treinamento com o dataset combinado
history = model.fit(train_combined_ds, validation_data=test_combined_ds, epochs=5)

In [ ]:
# Visualização dos Resultados
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.legend()
plt.title("Acurácia por época")
plt.xlabel("Épocas")
plt.ylabel("Acurácia")
plt.grid(True)
plt.show()

In [ ]:
# Testar o modelo com imagens externas
def test_external_image(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image_array = tf.keras.preprocessing.image.img_to_array(image) / 255.0
    image_array = tf.expand_dims(image_array, axis=0)
    prediction = model.predict(image_array)
    class_name = "Cachorro" if prediction[0][0] > 0.5 else "Gato"
    print(f"Predição: {class_name} (Confiança: {prediction[0][0]:.2f})")

# Exemplo de teste com imagem externa
test_external_image("path/to/your/image.jpg")  # Substitua pelo caminho da imagem